## CS310 Natural Language Processing
## Lab 9: Transformer

In this lab, we will implement the multihead attention with future masking for causal language modeling. 

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## T1. Implement Self-Attention for a Single Head

First, prepare the input data in shape $N\times d$

*Hint*: 
- Use `torch.randn` to generate a torch tensor in the correct shape.

在 Transformer 中，每个输入 token（比如一个词或子词）都会被编码成一个 d 维的向量（常见取值是 512 或 768）。而一个句子就是这些向量组成的矩阵，形状是 [N, d]，其中 N 是句子中 token 的数量。

In [12]:
N = 3
d = 512
torch.manual_seed(0)

### START YOUR CODE ###
X = torch.randn(N, d)
### END YOUR CODE ###


# Test 
assert isinstance(X, torch.Tensor)
print('X.size():', X.size())
print('X[:,0]:', X[:,0].data.numpy())

# You should expect to see the following results:
# X.shape: (3, 512)
# X[:,0]: [-1.1258398  -0.54607284 -1.0840825 ]

X.size(): torch.Size([3, 512])
X[:,0]: [-1.1258398  -0.54607284 -1.0840825 ]


Then, initialize weight matrices $W^Q$, $W^K$, and $W^V$. We assume they are for a single head, so $d_k=d_v=d$

Using $W^Q$ as an example
- First initialize an empty tensor `W_q` in the dimension of $d\times d_k$, using the `torch.empty()` function. Then initialize it with `nn.init.xavier_uniform_()`.
- After `W_q` is initialized, obtain the query matrix `Q` with a multiplication between `X` and `W_q`, using `torch.matmul()`.

In [13]:
torch.manual_seed(0) # Do not remove this line

n_heads = 1

### START YOUR CODE ###
d_k = d // n_heads # Compute d_k

# 初始化权重矩阵 W_q, W_k, W_v
W_q = torch.empty(d, d_k)
W_k = torch.empty(d, d_k)
W_v = torch.empty(d, d_k)

# 使用 Xavier 初始化
nn.init.xavier_normal_(W_q)
nn.init.xavier_normal_(W_k)
nn.init.xavier_normal_(W_v)

# 计算 Q, K, V
Q = torch.matmul(X, W_q)  # [N, d_k]
K = torch.matmul(X, W_k)  # [N, d_k]
V = torch.matmul(X, W_v)  # [N, d_k]
### END YOUR CODE ###


# Test
assert Q.size() == (N, d_k)
assert K.size() == (N, d_k)
assert V.size() == (N, d_k)

print('Q.size():', Q.size())
print('Q[:,0]:', Q[:,0].data.numpy())
print('K.size():', K.size())
print('K[:,0]:', K[:,0].data.numpy())
print('V.size():', V.size())
print('V[:,0]:', V[:,0].data.numpy())

# You should expect to see the following results:
# Q.size(): torch.Size([3, 512])
# Q[:,0]: [-0.45352045 -0.40904033  0.18985942]
# K.size(): torch.Size([3, 512])
# K[:,0]: [ 1.509987   -0.5503683   0.44788954]
# V.size(): torch.Size([3, 512])
# V[:,0]: [ 0.43034226  0.00162293 -0.1317436 ]

Q.size(): torch.Size([3, 512])
Q[:,0]: [-0.45352045 -0.40904033  0.18985942]
K.size(): torch.Size([3, 512])
K[:,0]: [ 1.509987   -0.5503683   0.44788954]
V.size(): torch.Size([3, 512])
V[:,0]: [ 0.43034226  0.00162293 -0.1317436 ]


Lastly, compute the attention scores $\alpha$ and the weighted output

Following the equation:
$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V
$$

*Hint*:
- $\alpha = \text{softmax}(\frac{QK^\top}{\sqrt{d_k}})$, where you can use `torch.nn.functional.softmax()` to compute the softmax. Pay attention to the `dim` parameter.
- The weighted output is the multiplication between $\alpha$ and $V$. Pay attention to their dimensions: $\alpha$ is of shape $N\times N$, and $\alpha_{ij}$ is the attention score from the $i$-th to the $j$-th word. 
- The weighted output is of shape $N\times d_v$, and here we assume $d_k=d_v$.

In [14]:
### START YOUR CODE ###
# Step 1: QK^T / sqrt(d_k)
scores = torch.matmul(Q, K.T) / (d_k ** 0.5)  # shape [N, N]

# Step 2: softmax to get alpha
alpha = F.softmax(scores, dim=1)  # shape [N, N]

# Step 3: weighted sum of V
output = torch.matmul(alpha, V)  # shape [N, d_v]
### END YOUR CODE ###


# Test
assert alpha.size() == (N, N)
assert output.size() == (N, d_k)

print('alpha.size():', alpha.size())
print('alpha:', alpha.data.numpy())
print('output.size():', output.size())
print('output[:,0]:', output[:,0].data.numpy())

# You should expect to see the following output:
# alpha.size(): torch.Size([3, 3])
# alpha: [[0.78344566 0.14102352 0.07553086]
#  [0.25583813 0.18030964 0.5638523 ]
#  [0.09271843 0.2767209  0.63056064]]
# output.size(): torch.Size([3, 512])
# output[:,0]: [ 0.32742795  0.03610666 -0.04272257]

alpha.size(): torch.Size([3, 3])
alpha: [[0.78344566 0.14102352 0.07553086]
 [0.25583813 0.18030964 0.5638523 ]
 [0.09271843 0.2767209  0.63056064]]
output.size(): torch.Size([3, 512])
output[:,0]: [ 0.32742795  0.03610666 -0.04272257]


## T2. Mask Future Tokens

First, create a binary mask tensor of size $N\times N$, which is lower triangular, with the diagonal and upper triangle set to 0.

*Hint*: Use `torch.tril` and `torch.ones`.

在实现 Transformer 的 decoder 部分时，我们需要一个 下三角掩码（mask），以防模型在训练过程中“偷看”未来的 token（实现 自回归）

In [15]:
### START YOUR CODE ###
mask = torch.tril(torch.ones(N, N))
### END YOUR CODE ###

# Test
print('mask:', mask.data.numpy())

# You should expect to see the following output:
# mask: [[1. 0. 0.]
#  [1. 1. 0.]
#  [1. 1. 1.]]

mask: [[1. 0. 0.]
 [1. 1. 0.]
 [1. 1. 1.]]


Use the mask to fill the corresponding future cells in $QK^\top$ with $-\infty$ (`-np.inf`), and then pass it to softmax to compute the new attention scores.

*Hint*: Use `torch.Tensor.masked_fill` function to selectively fill the upper triangle area of the result.

In [16]:
### START YOUR CODE ###
new_alpha = torch.nn.functional.softmax(
    scores.masked_fill(mask == 0, float('-inf')), dim=1
) # 将未来 token 的打分屏蔽为 -inf;对每一行做 softmax，即每个 token 对其它 token 的注意力分布
### END YOUR CODE ###


# Test
print('new_alpha:', new_alpha.data.numpy())

# You should expect to see the following results:
# new_alpha: [[1.         0.         0.        ]
#  [0.5865858  0.41341412 0.        ]
#  [0.09271843 0.2767209  0.63056064]]

new_alpha: [[1.         0.         0.        ]
 [0.5865858  0.41341412 0.        ]
 [0.09271843 0.2767209  0.63056064]]


Lastly, the output should also be updated:

In [17]:
### START YOUR CODE ###
new_output = torch.matmul(new_alpha, V)  # shape: [3, 512]
### END YOUR CODE ###

# Test
print('new_output.size():', new_output.size())
print('new_output[:,0]:', new_output[:,0].data.numpy())

# You should expect to see the following results:
# new_output.size(): torch.Size([3, 512])
# new_output[:,0]: [ 0.43034226  0.2531036  -0.04272257]

new_output.size(): torch.Size([3, 512])
new_output[:,0]: [ 0.43034226  0.2531036  -0.04272257]


## T3. Integrate Multiple Heads

Finally, integrate the above implemented functions into the `MultiHeadAttention` class.

**Note**:

- In this class, the weight matrices `W_q`, `W_k`, and `W_v` are defined as tensors of size $d\times d$. Thus, the output $Q=XW^Q$ is of size $N\times d$.

- Then we reshape $Q$ (and $K$, $V$ as well) into the tensor `Q_` of shape $N\times h\times d_k$, where $h$ is the number of heads (`n_heads`) and $d_k = d // h$. Similar operations are applied to $K$ and $V$. 

- The multiplication $QK^\top$ is now between two tensors of shape $N\times h\times d_k$, `Q_` and `K_`, and the output is of size $h\times N \times N$. Thus, you need to use `torch.matmul` and `torch.permute` properly to make the dimensions of `Q_`, `K_`, and `V_` be in the correct order.

- Also, remember to apply the future mask to each attention head's output. You can use `torch.repeat` to replicate the mask for `n_heads` times.

In [20]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_heads == 0
        self.d_k = d_model // n_heads
        self.n_heads = n_heads

        self.W_q = nn.Parameter(torch.empty(d_model, d_model))
        self.W_k = nn.Parameter(torch.empty(d_model, d_model))
        self.W_v = nn.Parameter(torch.empty(d_model, d_model))

        nn.init.xavier_normal_(self.W_q)
        nn.init.xavier_normal_(self.W_k)
        nn.init.xavier_normal_(self.W_v)
        
    def forward(self, X):
        N = X.size(0)
        
        ### START YOUR CODE ###
        # Step 1: QKV projection
        Q = torch.matmul(X, self.W_q)  # [N, d_model]
        K = torch.matmul(X, self.W_k)  # [N, d_model]
        V = torch.matmul(X, self.W_v)  # [N, d_model]

        # Step 2: reshape into multi-head format: [N, h, d_k]
        Q_ = Q.view(N, self.n_heads, self.d_k)
        K_ = K.view(N, self.n_heads, self.d_k)
        V_ = V.view(N, self.n_heads, self.d_k)

        # Step 3: permute to [h, N, d_k] for batched attention
        Q_ = Q_.permute(1, 0, 2)  # [h, N, d_k]
        K_ = K_.permute(1, 0, 2)
        V_ = V_.permute(1, 0, 2)
        
        # Raw attention scores
        scores = torch.matmul(Q_, K_.transpose(1, 2)) / (self.d_k ** 0.5)
        # Apply the mask
        mask = torch.tril(torch.ones(N, N, device=X.device)).bool()
        mask = mask.unsqueeze(0).repeat(self.n_heads, 1, 1)
        scores = scores.masked_fill(~mask, float('-inf'))

        # Softmax
        alpha = F.softmax(scores, dim=-1)

        # Weighted sum
        context = torch.matmul(alpha, V_)

        # Reshape back
        context = context.permute(1, 0, 2).contiguous()
        output = context.view(N, self.n_heads * self.d_k)
        ### END YOUR CODE ###

        return output

In [22]:
# Test
torch.manual_seed(0)

multi_head_attn = MultiHeadAttention(d, n_heads=1)
output = multi_head_attn(X)

assert output.size() == (N, d)
print('output.size():', output.size())
print('output[:,0]:', output[:,0].data.numpy())

# You should expect to see the following results:
# output.size(): torch.Size([3, 512])
# output[:,0]: [ 0.43034226  0.2531036  -0.04272257]

output.size(): torch.Size([3, 512])
output[:,0]: [ 0.43034226  0.2531036  -0.04272257]


**Note** that the above output size and values should be the same as the previous one, as we used `n_heads=1`.